# -------------------软采小区各地市覆盖评估----------------------
###   ：使用整理好的软采小区和栅格数据进行各地市的覆盖评估

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import mytools

## ----------导入软采小区数据----------

In [3]:
ruancai_cel = mytools.othern.pickle_read('G:/1-规划/评估/软采评估/2019第四季度/软采小区/使用摸底小区/存放数据/ruancai_jingdui.data')

In [4]:
day,city,cgi,cell_name,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak ,hd,wt_z= [
    'a.day', 'a.region_name', 'cgi', 'a.eutrancell_name','rsrp_count', 
    'rsrp_weak', 'a.un_maxrsrp_sample_count', 'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count', 'a.te_maxrsrp_weak_l2','hd','总问题小区']

### ---匹配工参字段---

In [5]:
gongcan = pd.read_csv(open('G:/1-规划/工参/工参数据/ltejizhan20200108.csv',encoding='gbk'),usecols=['小区CGI(*)','所属地区','归属区县','RRU经度','RRU纬度','厂家名称'])

In [6]:
gongcan_cgi,gongcan_city,gongcan_qx,gongcan_lon,gongcan_lat,changjia_m=['小区CGI(*)','所属地区','归属区县','RRU经度','RRU纬度','厂家名称']

In [7]:
gongcan.columns

Index(['小区CGI(*)', '所属地区', 'RRU经度', 'RRU纬度', '归属区县', '厂家名称'], dtype='object')

In [8]:
ruancai_cel = ruancai_cel.reset_index(drop=True).reset_index()

In [9]:
ruancai_cel_addgongcan = ruancai_cel.merge(gongcan,how='left',left_on=cgi,right_on=gongcan_cgi).drop_duplicates('index').drop(columns='index')

In [10]:
ruancai_cel_addgongcan_addchngjing = mytools.gisn.points_coverage_merge(ruancai_cel_addgongcan,
    lon=gongcan_lon,
    lat=gongcan_lat,
    coverage='G:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',
    df_merge_coverage_columns=['区域类型'],
    merge_only_right_mark='农村')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [11]:
quxian_xin = mytools.othern.pickle_read('G:/1-规划/图层/县城和地市归属图层pickle/县界.data')

In [12]:
ruancai_cel_qx = mytools.gisn.points_coverage_merge(ruancai_cel_addgongcan_addchngjing,
                                                                        lon=gongcan_lon,
                                                                        lat=gongcan_lat,
                                                                        coverage=quxian_xin,
                                                                        df_merge_coverage_columns=['城市','NAME'],
                                                                        merge_only_right_mark='农村')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [13]:
ruancai_use1 = ruancai_cel_qx.loc[(ruancai_cel_qx['城市'].notna()) & (ruancai_cel_qx['NAME'].notna())]

In [14]:
ruancai_use = ruancai_use1[['城市','NAME',rsrp_count,rsrp_weak,'区域类型',un_count,un_weak,te_count,te_weak ,hd,wt_z,changjia_m,'小区CGI(*)']]

In [15]:
ruancai_use['fgl'] = 1-ruancai_use[rsrp_weak]/ruancai_use[rsrp_count]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
ruancai_use.loc[ruancai_use['fgl']<0.9,'弱覆盖小区']=1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
mytools.othern.pickle_save(ruancai_use,'G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/ruancai_use.data')

### -----软采小区整理-全网指标-----

In [ ]:
#覆盖率

In [ ]:
ruancai_quanwang = ruancai_use.groupby('城市').agg({rsrp_count:'sum',rsrp_weak:'sum'})

In [ ]:
a = ruancai_quanwang.sum()
a.name='全网'
ruancai_quanwang = ruancai_quanwang.append(a)

In [ ]:
ruancai_quanwang['全网移动覆盖率'] = 1-ruancai_quanwang[rsrp_weak]/ruancai_quanwang[rsrp_count]

In [ ]:
ruancai_quanwnag_use = mytools.othern.df_sort(ruancai_quanwang.reset_index(),
    columns=['全网移动覆盖率'],
    sort_columns='全网移动覆盖率',
    ascending=False)

In [ ]:
ruancai_quanwnag_use.columns = ['全网-指标', '移动总点', '移动弱点', '全网移动覆盖率']

In [ ]:
ruancai_quanwnag_use.to_clipboard()

In [ ]:
mytools.othern.pickle_save(ruancai_quanwnag_use,'G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/dict_qw_fgl.data')

In [ ]:
#弱覆盖小区占比

In [ ]:
ruancai_use['fgl_cmcc'] = 1- ruancai_use[rsrp_weak]/ruancai_use[rsrp_count]

In [ ]:
ruancai_use[un_weak] = pd.to_numeric(ruancai_use[un_weak])
ruancai_use[un_count] = pd.to_numeric(ruancai_use[un_count])
ruancai_use[te_count] = pd.to_numeric(ruancai_use[te_count])
ruancai_use[te_weak] = pd.to_numeric(ruancai_use[te_weak])

In [ ]:
ruancai_use['fgl_un'] = 1- ruancai_use[un_weak]/ruancai_use[un_count]
ruancai_use['fgl_te'] = 1- ruancai_use[te_weak]/ruancai_use[te_count]

In [ ]:
#黑点打标
def hd(data,cmcc='',jingdui=''):
    data.loc[((data[cmcc]<0.8)&(data[jingdui]>0.8)) | (data[cmcc]>0.8)&(((data[jingdui])-(data[cmcc]))>0.05),'hd']=1
    return data['hd']

In [ ]:
ruancai_use['un_hd']=hd(ruancai_use,'fgl_cmcc','fgl_un')
ruancai_use['te_hd']=hd(ruancai_use,'fgl_cmcc','fgl_te')

In [ ]:
ruancai_use.loc[(ruancai_use['un_hd']==1) | (ruancai_use['te_hd']==1),'hd']=1

In [ ]:
cell_zb_qw = ruancai_use.groupby('城市').agg({'城市':'count','弱覆盖小区':'sum','hd':'sum'})

In [ ]:
cell_zb_qw['问题小区数'] = cell_zb_qw['弱覆盖小区'] +cell_zb_qw['hd'] 

cell_zb_qw['问题小区数'] = pd.to_numeric(cell_zb_qw['问题小区数'],downcast='integer')
cell_zb_qw['弱覆盖小区'] = pd.to_numeric(cell_zb_qw['弱覆盖小区'],downcast='integer')
cell_zb_qw['hd'] = pd.to_numeric(cell_zb_qw['hd'],downcast='integer')

In [ ]:
a = cell_zb_qw.sum()
a.name='全网'
cell_zb_qw_v2 = cell_zb_qw.append(a)

In [ ]:
cell_zb_qw_v2['问题小区占比'] = cell_zb_qw_v2['问题小区数']/cell_zb_qw_v2['城市']

In [ ]:
cell_zb_qw_v2.columns

In [ ]:
cell_zb_qw_v2.columns = ['总小区数', '弱覆盖小区数', '黑点小区数','问题小区数','问题小区占比']

In [ ]:
ruancai_zhanbi = mytools.othern.df_sort(cell_zb_qw_v2.reset_index(),
    columns=['问题小区占比'],
    sort_columns='问题小区占比',ascending=True)

In [ ]:
mytools.othern.pickle_save(ruancai_zhanbi,'G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/dict_qw_zb.data')

### -----软采小区整理-区县指标-----

In [ ]:
citys = ['孝感','襄阳','咸宁','随州','十堰','荆州','荆门','黄石','鄂州','黄冈','恩施','宜昌','潜江','天门','江汉','武汉']

In [ ]:
#覆盖率

In [ ]:
ruancai_use_qx_fg = ruancai_use[['城市','NAME',rsrp_count,rsrp_weak]].groupby(['城市','NAME']).agg({rsrp_count:'sum',rsrp_weak:'sum'})

In [ ]:
qx_fg_res = dict()
for city_t in citys:
    ruancai_use_qx_t = ruancai_use_qx_fg.loc[city_t,]
    a = ruancai_use_qx_t.sum()
    a.name='总计'
    ruancai_use_qx_t = ruancai_use_qx_t.append(a)
    ruancai_use_qx_t[rsrp_weak] = pd.to_numeric(ruancai_use_qx_t[rsrp_weak],downcast='integer')
    ruancai_use_qx_t[rsrp_count] = pd.to_numeric(ruancai_use_qx_t[rsrp_count],downcast='integer')

    ruancai_use_qx_t['移动覆盖率'] = 1-ruancai_use_qx_t[rsrp_weak]/ruancai_use_qx_t[rsrp_count]
    ruancai_use_qx_t.columns = ['移动总点', '移动弱点', '移动覆盖率']
    ruancai_use_qx_t2 = mytools.othern.df_sort(ruancai_use_qx_t.reset_index(),
        columns=['移动覆盖率'],
        sort_columns='移动覆盖率',
        ascending=False)
    
    ruancai_use_qx_t2 = ruancai_use_qx_t2.loc[ruancai_use_qx_t2['NAME']!='武汉市']
    ruancai_use_qx_t2.columns=['区域','移动总点', '移动弱点', '移动覆盖率']
    qx_fg_res[city_t] = ruancai_use_qx_t2


In [ ]:
mytools.othern.pickle_save(qx_fg_res,'G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/qx_fg_res.data')

In [ ]:
#弱覆盖小区占比

In [ ]:
ruancai_use_qx = ruancai_use[['城市','NAME','弱覆盖小区','hd']].groupby(['城市','NAME']).agg({'城市':'count','弱覆盖小区':'sum','hd':'sum'})

In [ ]:
ruancai_use_qx['问题小区数'] = ruancai_use_qx['弱覆盖小区'] +ruancai_use_qx['hd'] 

ruancai_use_qx['问题小区数'] = pd.to_numeric(ruancai_use_qx['问题小区数'],downcast='integer')
ruancai_use_qx['弱覆盖小区'] = pd.to_numeric(ruancai_use_qx['弱覆盖小区'],downcast='integer')
ruancai_use_qx['hd'] = pd.to_numeric(ruancai_use_qx['hd'],downcast='integer')

In [ ]:
ruancai_use_qx_t

In [ ]:
qx_res = dict()
for city_t in citys:
    ruancai_use_qx_t = ruancai_use_qx.loc[city_t,]
    a = ruancai_use_qx_t.sum()
    a.name='总计'
    ruancai_use_qx_t = ruancai_use_qx_t.append(a)
    ruancai_use_qx_t['城市'] = pd.to_numeric(ruancai_use_qx_t['城市'],downcast='integer')
    ruancai_use_qx_t['弱覆盖小区'] = pd.to_numeric(ruancai_use_qx_t['弱覆盖小区'],downcast='integer')

    ruancai_use_qx_t['问题小区占比'] = ruancai_use_qx_t['问题小区数']/ruancai_use_qx_t['城市']
    ruancai_use_qx_t.columns = ['总小区数', '弱覆盖小区数', '黑点小区数','问题小区数','问题小区占比']
    ruancai_use_qx_t2 = mytools.othern.df_sort(ruancai_use_qx_t.reset_index(),
        columns=['问题小区占比'],
        sort_columns='问题小区占比',
        ascending=True)
    ruancai_use_qx_t2.columns=['区域','总小区数', '弱覆盖小区数', '黑点小区数','问题小区数','问题小区占比']
    qx_res[city_t] = ruancai_use_qx_t2

In [ ]:
mytools.othern.pickle_save(qx_res,'G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/dict_qx.data')

# ----------导入栅格数据----------

In [ ]:
grid_changjing = pd.read_csv(open('g:/1-规划/评估/软采评估/2019第四季度/栅格/1千500万个栅格11和12月汇总_去重.csv',encoding='utf-8'),chunksize=100000)

In [ ]:
day,city,grid_id,lon,lat,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak,changjing = ['a.day', 'a.region_name', 'a.grid_id', 'lon', 'lat',
       'a.rsrp_sample_count', 'a.rsrp_weak', 'a.un_maxrsrp_sum',
       'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count',
       'a.te_maxrsrp_weak_l2', '区域类型']

In [ ]:
quxian_xin = mytools.othern.pickle_read('G:/1-规划/图层/县城和地市归属图层pickle/县界.data')

In [ ]:
data_z = pd.DataFrame()
for data_t in grid_changjing:
    data_t_use_v2 = mytools.gisn.points_coverage_merge(data_t,lon,lat,coverage=quxian_xin,
                                                        df_merge_coverage_columns=['城市','NAME'],
                                                        merge_only_right_mark='农村')
    data_t_use_v2_1 = data_t_use_v2.loc[(data_t_use_v2['城市'].notna()) & (data_t_use_v2['NAME'].notna())]
    data_z = data_z.append(data_t_use_v2_1)
    print(data_t.shape,data_z.shape)

In [ ]:
data_z1 = data_z.reset_index(drop=True).reset_index()

In [ ]:
data_z1.loc[data_z1[rsrp_count]>100,'有效栅格']=1
data_z1['fgl']=1-data_z1[rsrp_weak]/data_z1[rsrp_count]
data_z1.loc[(data_z1['fgl']<0.936)&(data_z1['有效栅格']==1),'弱覆盖栅格数量']=1
data_z1.loc[(data_z1['fgl']<0.7) &(data_z1['有效栅格']==1),'严重弱覆盖栅格数量']=1

In [ ]:
mytools.othern.pickle_save(data_z1,'g:/1-规划/评估/软采评估/2019第四季度/栅格/grid_add_yanzhong.data')

In [ ]:
data_groupby_city = data_z1.groupby('城市')

In [ ]:
grid_dict = dict()
for name,data_t in data_groupby_city:
    print(name)
    data_res = data_t.groupby('NAME').agg({grid_id:'count','有效栅格':'sum','弱覆盖栅格数量':'sum','严重弱覆盖栅格数量':'sum'})
    data_res_v2 = data_res.reset_index()
    data_res_v2.columns=['2019年12月','总栅格','大于100采样点','弱覆盖栅格数量','严重弱覆盖栅格数量']
    grid_dict[name] = data_res_v2

In [ ]:
data_res = data_z1.groupby('城市').agg({grid_id:'count','有效栅格':'sum','弱覆盖栅格数量':'sum','严重弱覆盖栅格数量':'sum'})
data_res_v2 = data_res.reset_index()
data_res_v2.columns=['2019年12月','总栅格','大于100采样点','弱覆盖栅格数量','严重弱覆盖栅格数量']
grid_dict['全网'] = data_res_v2

In [ ]:
mytools.othern.pickle_save(grid_dict,'g:/1-规划/评估/软采评估/2019第四季度/栅格/保存数据/dict_grid.data')